RAM이 부족해서 안됨..data set을 randaom으로 생성한 뒤 저장하고 해보기

In [104]:
clear

In [105]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [106]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# 시간정보를 활용하여 폴더 생성
import datetime 

In [107]:
#3채널 525*700 픽셀의 500개 이미지
IMG_CHANNELS = 3
IMG_ROWS = 525
IMG_COLS = 700
IMG_SIZE = (IMG_ROWS, IMG_COLS)

In [118]:
VALIDATION_SPLIT = 0.10

img_gen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split= VALIDATION_SPLIT)
#img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=VALIDATION_SPLIT) 

train_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms",
                                        target_size = IMG_SIZE,                                                     
                                        class_mode='categorical',
                                        subset='training',
                                        shuffle=None, 
                                        batch_size = 500)

test_set = img_gen.flow_from_directory(directory="/content/drive/MyDrive/dataset/Generated_Spectograms/",
                                        target_size = IMG_SIZE,
                                        class_mode='categorical',
                                        subset='validation',
                                        shuffle=None,
                                        batch_size = 500)

# 이미지와 이미지에 해당하는 label
Train_X, Train_Y = next(train_set)
Test_X, Test_Y = next(test_set)

Found 450 images belonging to 5 classes.
Found 50 images belonging to 5 classes.


In [119]:
# 그래프만 추출
X_train = Train_X[:, 39:467, 81:567, :]
X_test = Test_X[:, 39:467, 81:567, :]

In [ ]:
# image augmentation: random_crop
def random_crop(img_set, random_crop_width):
    height, width = img_set.shape[1], img_set.shape[2]
    dx = random_crop_width
    x = np.random.randint(0, width - dx + 1)
    return img_set[ :, :, x:(x+dx), :]

In [ ]:
crop_width = 32

argumented_X_train = random_crop(X_train, crop_width)
argumented_Y_train = Train_Y

for i in range(9):
  new_set = random_crop(X_train, crop_width)
  argumented_X_train = np.concatenate((argumented_X_train, new_set),axis=0)
  argumented_Y_train = np.concatenate((argumented_Y_train,Train_Y),axis=0)

argumented_X_test = random_crop(X_test, crop_width)
argumented_Y_test = Test_Y

for i in range(9):
  new_set = random_crop(X_test, crop_width)
  argumented_X_test = np.concatenate((argumented_X_test, new_set),axis=0)
  argumented_Y_test = np.concatenate((argumented_Y_test, Test_Y),axis=0)


In [ ]:
argumented_X_train = tf.image.resize(argumented_X_train, (428, 81))
argumented_X_test = tf.image.resize(argumented_X_test, (428, 81))

# 형식 변환
argumented_X_train = argumented_X_train.numpy().astype('float32')
argumented_X_test = argumented_X_test.numpy().astype('float32')


https://colab.research.google.com/github/ndb796/Python-Data-Analysis-and-Image-Processing-Tutorial/blob/master/05.%20Numpy%EC%9D%98%20%ED%99%9C%EC%9A%A9/Numpy%EC%9D%98%20%ED%99%9C%EC%9A%A9.ipynb#scrollTo=cJ9HX7GtL6tT

In [ ]:
# 상수
BATCH_SIZE =  100
EPOCHS = 100
NUM_CLASSES = 5
VERBOSE = 1
OPTIM = tf.keras.optimizers.RMSprop()

INPUT_SHAPE = (428, 81, 3)

In [ ]:
from keras.applications import inception_v3
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

base_model = tf.keras.applications.inception_v3.InceptionV3(
    include_top= False, weights= None, input_shape=INPUT_SHAPE, classes = NUM_CLASSES
    #classifier_activation='softmax'
)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation = "relu")(x)
predictions = Dense(5, activation = 'softmax')(x)

#model.add로 하는 방법..

model = Model(inputs = base_model.input, outputs = predictions)
model.summary()


In [ ]:
# 텐서보드 사용
callbacks = [
    # 텐서보드 로그를 './logs' 디렉터리에 작성
    tf.keras.callbacks.TensorBoard(log_dir = './logs')
]

# 학습데이터의 log를 저장할 폴더 생성 (지정)
log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 텐서보드 콜백 정의 하기
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
#https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
#https://lynnshin.tistory.com/23
from sklearn.model_selection import KFold

# Define per-fold score containers
VALIDATION_ACCURACY = []
VALIDATIOM_LOSS = []

num_folds = 10
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(argumented_X_train, argumented_Y_train):
    # 모델 초기화 및 컴파일
    training_data = argumented_X_train[train]
    training_lable = argumented_Y_train[train]
    validation_data = argumented_X_train[test]
    validation_lable = argumented_Y_train[test]

    model = Model(inputs = base_model.input, outputs = predictions)
    model.compile(loss = "categorical_crossentropy", optimizer = OPTIM, metrics = ["accuracy"])

    # 학습데이터의 log를 저장할 폴더 생성 (지정)
    log_dir = "logs/my_board/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  
   # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    history = model.fit(training_data, training_lable, 
                        batch_size = BATCH_SIZE, epochs = EPOCHS, verbose = 1,
                        validation_data = (validation_data, validation_lable))
   # Generate generalization metrics
    scores = model.evaluate(argumented_X_test, argumented_Y_test, verbose = 0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    VALIDATION_ACCURACY.append(scores[1] * 100)
    VALIDATIOM_LOSS.append(scores[0])

    
    # 모델 저장
    model_title = "/content/drive/MyDrive/model/coughdetection/inception_V3_ " + str(fold_no) + ".json"
    weights_title = '/content/drive/MyDrive/model/coughdetection/inception_V3_weights_' + str(fold_no) + ".json"

    model_json = model.to_json()
    with open(model_title, 'w') as json_file:
      json_file.write(model_json)
    model.save_weights(weights_title)

   # Increase fold number
    fold_no = fold_no + 1

In [ ]:

argumented_X_test = tf.image.resize(argumented_X_test, (428, 81))
score = model.evaluate(argumented_X_test, argumented_Y_test, batch_size = BATCH_SIZE)
print("\nTest score:", score[0])
print("Test accuracy:", score[1])

In [ ]:
print(type(argumented_X_train))
print(argumented_X_train.shape)